In [1]:
#Import Libraries

import Data_manipulation_functions as dmf
import pandas as pd
import numpy as np
import os
from pathlib import Path

#Set path

data_inputs= '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input'

# Raport: Badanie optymalnej wyceny piłkarza.



## 1. Wstęp

Rynek transferów piłkarskich w ostatnich latach przeżył niesamowity rozkwit, stając się bardzo dochodowym biznesem dla klubów. Do 2008 roku najdroższym transferem w historii było przejście Zinedine Zidane’a z Juventusu do Realu Madryt w 2001 roku. Królewscy zapłacili za piłkarza „Starej Damy” około 47 mln euro. Zidane był uważany wówczas za najlepszego na świecie, mając status na poły legendarny. Obecnie kluby wydają kwoty rzędu 50 mln euro, aby sprowadzić byle gwiazdkę, juniora z ligi brazylijskiej (np. 18 letni Victor Roque kosztował Barcelonę FC 40 mln euro w 2024 roku). W związku z tym niezwykle istotnym stało się właściwe wycenianie piłkarzy, zarówno z perspektywy kupującego, jak i sprzedającego klubu, ponieważ stali się oni niemalże dobrem inwestycyjnym, które może przynieść zwrot z późniejszej sprzedaży, albo przynieść określone zyski ze zdobycia tytułu mistrzowskiego.

Niniejszy raport został stworzony w celu zbadania optymalnej wyceny piłkarzy w ligach tzw. top 5, do których należą najwyższe klasy rozgrywkowe w Anglii, Hiszpanii, Francji, Niemczech i Włoszech. Jego głównym zadaniem jest wybór modelu uczenia maszynowego, który najlepiej przewiduje potencjalną cenę sprzedaży piłkarza na podstawie jego indywidualnych statystyk oraz innych zmiennych, które mogą mieć wpływ na ostateczną wycenę.

## 2. Dane

Aby dotrzeć do jak najbardziej wiarygodnych danych, do stworzenia raportu wykorzystano bazę danych serwisu Transfermarkt.de, który zbiera informacje dotyczące rynku transferowego, często używane w mediach. Niestety, portal nie oferuje API, które mogłoby posłużyć jako punkt dostępu do bazy danych, dlatego zastosowano API utworzone przez jednego z użytkowników Github.com, dostępne pod adresem: https://github.com/felipeall/transfermarkt-api?tab=readme-ov-file.

W oparciu o ten punkt dostępu stworzono dwie klasy: Clubs i Players, które są dostępne w pliku DataSource.py. Ich zadaniem jest odczytywanie, przetwarzanie i zapisywanie danych pochodzących z Transfermarkt.de do formatu pandas DataFrame.

Następnym krokiem jest utworzenie instancji klasy Players i zapisanie ich jako plików .csv, co ułatwi ich odczyt. Cały kod użyty do tych operacji znajduje się w pliku Data_sourcing_scripts.py. Ze względu na wykorzystanie synchronicznej metody dostępu do portalu Transfermarkt.de — wymuszonej przez sam serwis, który blokuje nadmierną ilość żądań — skrypty czytające dane należy uruchamiać tylko w przypadku istotnych zmian, takich jak dodanie nowych sezonów do zakresu.


In [2]:
path= os.getcwd()
directory_path = Path(path)

In [3]:
directory_path

PosixPath('/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python')

In [2]:
check= dmf.retrieve_directories(path)

In [3]:
check

['England', 'France', 'Spain', 'Germany', 'Italy']

In [3]:
df= dmf.raw_df(path)
#df= dmf.data_cleansing(df)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/England/P'

In [4]:
df.columns

Index(['Unnamed: 0_x', 'id', 'from', 'to', 'date', 'upcoming', 'season',
       'marketValue_x', 'fee', 'Unnamed: 0_y', 'name', 'position',
       'dateOfBirth', 'age', 'nationality', 'currentClub', 'height', 'foot',
       'joinedOn', 'signedFrom', 'marketValue_y', 'status', 'joined',
       'contract', 'Unnamed: 0', 'competitionID', 'clubID', 'seasonID',
       'competitionName', 'appearances', 'yellowCards', 'goalsConceded',
       'cleanSheets', 'minutesPlayed', 'redCards', 'goals', 'assists',
       'secondYellowCards'],
      dtype='object')

In [7]:
def adding_competitions(df, ref_data):
    ref_data= pd.read_csv(ref_data)
    competitions_dict= {'GB1': 'England', 'FR1': 'France', 'ES1':'Spain', 'IT1': 'Italy', 'L1':'Germany'}
    df= df.merge(ref_data, how= 'left', left_on= 'from', right_on= 'id')
    df['Competitions']= df['Competitions'].map(competitions_dict)
    df['Competitions']= df['Competitions'].fillna('Other')
    return df

['/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/P',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/r',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/o',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/v',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/i',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/d',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/e',
 '/Users/michalchruszczewski/Documents/Inżynieria Danych- Big Data/Projekt końcowy/Zadanie python/Data/Input/Italy/d',
 '/Users/michalchruszczewski/Documents/I

## x. Bibliografia

1. https://en.wikipedia.org/wiki/List_of_most_expensive_association_football_transfers (dostęp: 17.04.2024, godz. 12:40)
2. https://www.transfermarkt.pl/transfers/transfertagedetail/statistik/top/land_id_zu/0/land_id_ab/0/leihe//datum/2024-01-01 (dostęp: 17.04.2024, godz. 12:40)

1. Wstęp
Cel badania: Wyjaśnij, co dokładnie chcesz osiągnąć za pomocą swojego modelu przewidywania i dlaczego jest to istotne.
Tło badania: Krótkie wprowadzenie do tematu, w tym kontekst dotyczący piłki nożnej i wartości piłkarzy.
Pytania badawcze: Określ pytania, na które zamierzasz odpowiedzieć za pomocą swojego modelu przewidywania.
2. Przegląd literatury
Przegląd istniejących badań: Omówienie dotychczasowych badań i metod używanych do przewidywania wartości piłkarzy oraz jakie były ich wyniki.
3. Dane
Opis zbioru danych: Opisz skąd pochodzą Twoje dane, jakie zawierają zmienne, ile jest obserwacji itp.
Wstępne przetwarzanie danych: Opisz jakie kroki podjąłeś aby przygotować dane do analizy (czyszczenie danych, transformacja zmiennych, itd.).
4. Metodologia
Wybór metod: Opisz jakie techniki statystyczne lub algorytmy uczenia maszynowego planujesz użyć do budowy modelu przewidywania wartości piłkarzy.
Walidacja modelu: Opisz metody, które zastosujesz do oceny skuteczności modelu (np. walidacja krzyżowa, metryki oceny jak MAE, RMSE).
5. Wyniki
Prezentacja wyników modelowania: Przedstaw wyniki modeli, które zbudowałeś, włącznie z interpretacją współczynników modelu (jeśli to modele regresyjne).
Analiza wyników: Omów, które zmienne miały największy wpływ na wartość piłkarzy i jak model radzi sobie z przewidywaniami.
6. Dyskusja
Interpretacja wyników: Jak wyniki wpisują się w kontekst istniejących badań? Czy są jakieś niespodzianki lub oczekiwane wyniki?
Ograniczenia badania: Jakie są ograniczenia Twojego badania i modelu? Co mogłoby być zrobione inaczej?
7. Wnioski
Podsumowanie głównych odkryć: Krótkie streszczenie najważniejszych wyników i odpowiedzi na pytania badawcze.
Praktyczne zastosowania: Jak wyniki mogą być użyte w praktyce, np. przez kluby piłkarskie, agencje reprezentujące piłkarzy, media sportowe, itp.
Kierunki przyszłych badań: Sugestie dotyczące tego, jakie inne aspekty wartości piłkarzy mogłyby być badane w przyszłości.
8. Bibliografia
Źródła: Lista wszystkich cytowanych źródeł naukowych, raportów i innych materiałów, które wykorzystałeś w swojej pracy.
9. Załączniki / Dodatki
Kod źródłowy: Załącz kod użyty do analizy danych i budowy modelu.
Dodatkowe wykresy i tabelki: Materiały, które wspierają Twoją analizę, ale nie zostały włączone głównie do tekstu.

In [6]:
ref_data= pd.read_csv('Data/Input/Other/clubs_md.csv')

In [11]:
ref_data

,Unnamed: 0,id,name,Competitions
0,0,281,Manchester City,GB1
1,1,11,Arsenal FC,GB1
2,2,631,Chelsea FC,GB1
3,3,985,Manchester United,GB1
4,4,31,Liverpool FC,GB1
...,...,...,...,...
106,18,1531,Elche CF,ES1
107,19,2687,Cádiz CF,ES1
108,20,1108,Deportivo Alavés,ES1
109,21,472,UD Las Palmas,ES1


In [13]:
df= df.merge(ref_data, how= 'left', left_on= 'from', right_on= 'id')

In [15]:
df.fillna('Other')

,id_x,from,to,transferDate,marketValue,fee,position,dateOfBirth,age,nationality,...,cleanSheets,redCards,goals,assists,secondYellowCards,ageTransfer,Unnamed: 0,id_y,name,Competitions
0,2857,504,932,2006.0,750000.0,2000000.0,Goalkeeper,1984.0,38,Switzerland,...,2.0,0.0,0.0,0.0,0.0,22.0,Other,Other,Other,Other
1,2857,504,932,2006.0,750000.0,2000000.0,Goalkeeper,1984.0,38,Switzerland,...,15.0,0.0,0.0,0.0,0.0,22.0,Other,Other,Other,Other
2,2857,3008,1003,2017.0,1000000.0,2300000.0,Goalkeeper,1984.0,38,Switzerland,...,2.0,0.0,0.0,0.0,0.0,33.0,Other,Other,Other,Other
3,2857,3008,1003,2017.0,1000000.0,2300000.0,Goalkeeper,1984.0,38,Switzerland,...,90.0,3.0,1.0,0.0,0.0,33.0,Other,Other,Other,Other
4,3333,405,281,2010.0,20000000.0,22000000.0,Central Midfield,1986.0,37,England,...,0.0,0.0,0.0,4.0,0.0,24.0,8.0,405.0,Aston Villa,GB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10268,982855,417,276,2024.0,200000.0,500000.0,Defensive Midfield,2004.0,19,France,...,0.0,0.0,3.0,0.0,1.0,20.0,5.0,417.0,OGC Nice,FR1
10269,982855,417,276,2024.0,200000.0,500000.0,Defensive Midfield,2004.0,19,France,...,0.0,0.0,0.0,0.0,0.0,20.0,5.0,417.0,OGC Nice,FR1
10270,983989,1418,1237,2023.0,9000000.0,6000000.0,Attacking Midfield,2004.0,18,Argentina,...,0.0,0.0,20.0,12.0,0.0,19.0,Other,Other,Other,Other
10271,1038821,4655,11204,2023.0,150000.0,500000.0,Centre-Back,2004.0,19,Senegal,...,0.0,0.0,0.0,0.0,0.0,19.0,Other,Other,Other,Other
